In [1]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Project")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

sdf = spark.read.parquet("../data/curated/process_data.parquet/")

cols = ['merchant_name', 'user_id', 'dollar_value', 'order_datetime', 'rate', 'category', 'subcategory', 'estimated_region_population_2021_sum',	'persons_earners_2018-19_sum', 'mean_earnings_2018-19_avg', 'sum_earnings_2018-19_sum',	'median_earnings_2018-19_avg', 'med_age_earners_2018-19_avg']
sdf = sdf.select(cols)

sdf = sdf.withColumn('rate', F.col('rate') / 100)

sdf.limit(10)

22/10/06 16:23:19 WARN Utils: Your hostname, Shromanns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.12.144.226 instead (on interface en0)
22/10/06 16:23:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 16:23:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 16:23:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


merchant_name,user_id,dollar_value,order_datetime,rate,category,subcategory,estimated_region_population_2021_sum,persons_earners_2018-19_sum,mean_earnings_2018-19_avg,sum_earnings_2018-19_sum,median_earnings_2018-19_avg,med_age_earners_2018-19_avg
Placerat Orci Ins...,1,366.23,2021-11-17,0.0273,retail_and_wholes...,others_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Adipiscing Elit C...,1,113.59,2022-06-28,0.056600000000000004,info_media_and_te...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Quam Curabitur Co...,1,587.4,2021-09-24,0.018600000000000002,retail_and_wholes...,department_stores,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Felis Institute,1,51.58,2021-11-26,0.0642,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Class Aptent Taci...,1,196.41,2022-05-22,0.0501,info_media_and_te...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Augue Malesuada LLP,1,7.84,2022-08-10,0.0599,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Est Nunc Ullamcor...,1,100.22,2022-10-24,0.0572,retail_and_wholes...,household_goods_r...,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Iaculis Aliquet D...,1,115.82,2021-11-13,0.0176,retail_and_wholes...,food_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Iaculis Enim Corp.,1,18.5,2022-09-03,0.0381,rental_hiring_and...,null,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0
Iaculis Aliquet D...,1,63.88,2022-10-13,0.0176,retail_and_wholes...,food_retailing,10636,5795.0,62144.0,3.60125906E8,52618.0,41.0


### 1. **Identify** Merchants with the greatest gross and lowest volatility in income and the greatest evidence of performance

In [2]:
fullSdf = sdf.groupBy('merchant_name').agg(
        F.sum('dollar_value').alias('income_total'),
        F.stddev('dollar_value').alias('income_deviation'),
        F.mean('rate').alias('rate'),
        F.count('merchant_name').alias('count_merchant'))\
    .withColumn('income_total', F.col('income_total') * F.col('rate'))\
    .withColumn('income_deviation', F.col('income_deviation') * F.col('rate'))\
    .orderBy(F.col('count_merchant').desc(), F.col('income_total').desc(), F.col('income_deviation').asc())

In [3]:
days = sdf.select('order_datetime').distinct().count()

### 2. **Ranking** Merchants by Categories

In [4]:
def topNmerchants(sdf, categories, N):
    merchants = {}
    income_total = {}
    income_risk = {}
    counts = {}

    for category in categories:
        tdf = sdf.where(F.col('category') == category)
        tdf = tdf.groupBy('merchant_name').agg(
            F.sum('dollar_value').alias('income_total'),
            F.stddev('dollar_value').alias('income_deviation'),
            F.mean('rate').alias('rate'),
            F.count('merchant_name').alias('count_merchant'))\
        .withColumn('income_total', F.col('income_total') * F.col('rate'))\
        .withColumn('income_deviation', F.col('income_deviation') * F.col('rate'))\
        .orderBy(F.col('count_merchant').desc(), F.col('income_total').desc(), F.col('income_deviation').asc())\
        
        tdf = tdf.limit(N)

        merchants[category] = tdf.select('merchant_name').toPandas()['merchant_name'].to_list()
        
        # assuming that all income is Normally distributed
        income_total[category] = tdf.agg({'income_total': 'sum'}).first()['sum(income_total)']
        income_risk[category] = tdf.agg({'income_deviation': 'sum'}).first()['sum(income_deviation)']
        counts[category] = tdf.agg({'count_merchant': 'sum'}).first()['sum(count_merchant)']

    return pd.DataFrame(merchants), income_total, income_risk, counts

In [5]:
categories = ['retail_and_wholesale_trade', 'rental_hiring_and_real_estate', 'arts_and_recreation', 'info_media_and_telecommunications', 'others']

In [6]:
merchants, total, risk, counts = topNmerchants(sdf, categories, N=10)
merchants

,retail_and_wholesale_trade,rental_hiring_and_real_estate,arts_and_recreation,info_media_and_telecommunications,others
0,Erat Vitae LLP,Quis Massa Mauris Corporation,Magna Sed Industries,Mauris Non Institute,Nec Tellus Ltd
1,Leo In Consulting,Quam A Felis Incorporated,Ac Urna Consulting,Euismod In Corp.,Tempus Eu Ligula Limited
2,Pede Nonummy Corp.,Vel Lectus Cum LLC,Mi Consulting,Amet Consulting,Sed Nec Inc.
3,Non Vestibulum Industries,Mi Eleifend Company,Lorem LLP,Feugiat Sed Nec Institute,Natoque Consulting
4,Suspendisse Dui Corporation,Tellus Imperdiet Non Inc.,Volutpat Ornare Facilisis Associates,Arcu Sed Eu Incorporated,Gravida Praesent Corp.
5,Lacus Consulting,Iaculis Enim Corp.,Ligula Tortor Incorporated,Eleifend PC,Mauris Sagittis Corp.
6,Est Nunc Consulting,Leo Morbi Limited,Massa Suspendisse Corp.,Et Nunc Consulting,Nisl Elementum Ltd
7,Lorem Ipsum Sodales Industries,Laoreet Inc.,Turpis Nulla Foundation,Posuere Cubilia Curae Corporation,Molestie Arcu Corporation
8,Ipsum Dolor Sit Corporation,Massa Limited,Elit Curabitur Sed PC,At Sem Corp.,Feugiat Lorem Incorporated
9,Vehicula Pellentesque Corporation,Fermentum Institute,Nullam Scelerisque Ltd,Suspendisse Incorporated,Risus Odio Auctor Foundation


In [57]:
merchants.to_csv('./merchants.csv', sep=' ', index=False)

In [7]:
total

{'retail_and_wholesale_trade': 3624337.2294479636,
 'rental_hiring_and_real_estate': 354135.0366890002,
 'arts_and_recreation': 921992.9875999995,
 'info_media_and_telecommunications': 2296922.6012689797,
 'others': 707901.3082929963}

In [8]:
risk

{'retail_and_wholesale_trade': 11.665155832760831,
 'rental_hiring_and_real_estate': 303.35071177507166,
 'arts_and_recreation': 540.7637435210465,
 'info_media_and_telecommunications': 38.4802816812951,
 'others': 58.81002537036933}

In [9]:
counts

{'retail_and_wholesale_trade': 2328642,
 'rental_hiring_and_real_estate': 18540,
 'arts_and_recreation': 12759,
 'info_media_and_telecommunications': 454654,
 'others': 92509}

In [13]:
mu = sum(total.values()) / days
sigma = sum(risk.values()) / days

print('Average Daily Income:')
print(f'({mu - (3 * sigma)} <- 3 std - | {mu} | - 3 std -> {mu + (3 * sigma)})')

Average Daily Income:
(13040.313454693725 <- 3 std - | 13045.031622605511 | - 3 std -> 13049.749790517297)


22/10/06 17:44:55 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 904418 ms exceeds timeout 120000 ms
22/10/06 17:44:55 WARN SparkContext: Killing executors is not supported by current scheduler.
